In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statistics
src_path = r"D:/Codes/GIthub_repo/Energy_grid/data/UCSD_raw_data/PV_ALL.csv"

from XGBoost_helper import Data_encoder,Optuna_core,XGBoost_Optuna,Simple_forecast,intersection_sum

l:\Softwares\Anaconda\envs\gurobi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_prediction_Simple(src_path,start_t,end_t,days_ahead,save_path,K,alpha):
    
    data=pd.read_csv(src_path)
    data=data.drop_duplicates()
    data=data.sort_values(by="DateTime")
    data["DateTime"]=pd.to_datetime(data["DateTime"])
    data=data.set_index("DateTime")
    pred=data[start_t:end_t]
    pred=pred.sort_values(by="DateTime")

    #helper dataframe
    rng = pd.date_range(start=pred.index.min(),end=pred.index.max(), freq = '15min')
    pd_rng=pd.DataFrame(rng)
    pd_rng=pd_rng.rename(columns={0:'DateTime'}).copy()

    data_n=pd.merge(pred,pd_rng,on='DateTime',how='right')
    data_n['RealPower_pred'] = data_n.apply(lambda x: [], axis=1)
    data_n['DateTime']=pd.to_datetime(data_n['DateTime'])
    data_n=data_n.set_index('DateTime')
    data_n['RealPower']=data_n['RealPower'].astype(float)
    for i in data_n.index:
        if not isinstance(data_n.at[i,'RealPower'],float):
            data_n.at[i,'RealPower']=None
    
    data_n['historical_ave']=None
    for i in data_n.index:
        history=[]
        for k in range(days_ahead):
            index_ahead=i-timedelta(days=int(k+1))
            if index_ahead in data_n.index:
                history.append(data_n.at[index_ahead,'RealPower'])
            #else:
                #history.append(Null)                 
        ave=0
        count=0
        for m in history:
            if isinstance(m,(str,float,int)):
                ave=ave+float(m)
                count=count+1
        if count>0:
            ave=ave/count
            data_n.at[i,'historical_ave']=ave
    data_n=data_n.reset_index()
    
    last_weights = np.exp(- alpha*(np.arange(K)+1)) if alpha is not None else 0

    if 'ReactivePower' in data_n.columns:
        data_n=data_n.drop(columns=['ReactivePower'])
    print(data_n.head(50))
    
    for n in range(len(data_n)-K-1):
        # "realpower" 1st column
        last=data_n.iat[n,1]
        if not isinstance(last,float):
            last=None
        #last_weights = np.exp(- alpha*(np.arange(K)+1)) if alpha is not None else 0
        for i in range(K):
            #"historical_ave" 3rd column, 'Pred' 2nd column
            share=last_weights[i].copy()
            if not isinstance(share,float):
                print(last_weights)
                print(type(last_weights))
                raise Exception('invalid type of last_weights')
            if not isinstance(last,(float,None)):
                print(last)
                print(type(last))
                raise Exception('invalid type of last')  
            if  data_n.iat[n+i,3] != None:       
                if last != None:
                    pred_value=share*last+(1-share)*data_n.iat[n+i,3]
                else:
                    pred_value=data_n.iat[n+i,3]
                data_n.iat[n+i,2].append(pred_value)
    
    data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))
    data_n['pred_error']=data_n[["RealPower","RealPower_pred"]].\
        apply(lambda x: abs(x["RealPower"]-x["RealPower_pred"]),axis=1)
    
    ''' OLDer version of code:
    
    #pred=pred.set_index("DateTime")
    #pred.index = pd.to_datetime(pred.index)
    pred["RealPower_pred"]=None
    for i in pred.index:
        history=[]
        for k in range(days_ahead):
            index_ahead=i-timedelta(days=k+1)
            if index_ahead in pred.index:
                history.append(pred.loc[index_ahead,'RealPower'])
            else:
                history.append(pred.loc[i,'RealPower'])
        
        pred.loc[i,'RealPower_pred']=np.mean(history)
    '''
    data_n.to_csv(save_path,index=True)
    return data_n

In [3]:
metrics=pd.DataFrame(columns=['days_ahead','exp_alpha','MAE_1','MAE_4','MAE_24','MAE_48','MAE_72','MAE_96'])
for days in [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]:
    for alpha in [0.1,0.2,0.3,0.4,0.5]:
        save_path=r'L:\Coding_project\Energy_grid_new_exp_local\PV_simple_grid_search\PV_sum_Simple_prediction_'+str(days)+'days_'+str(alpha)+'.csv'
        Simple_pv=Simple_forecast(
            src_path=r'L:\Coding_project\Energy_grid_new\data\UCSD_raw_data\PV_Sum.csv',
            ref_start_t='30-10-2018',
            pred_start_t='1-1-2019',
            end_t='12-31-2019',
            method='ave_by_prediction_time',
            save_path=save_path,
            frequent='day',
            n=days,
            alpha=alpha,
            K=96,
            metrics_steps=[1,4,24,48,72,96]
        )
        Simple_pv.pred=Simple_pv.pred.drop(columns=['future_real','RealPower_pred'])
        Simple_pv.save_prediction()
        metrics.loc[len(metrics.index)] = [days,alpha,Simple_pv.pred["MAE_1"].mean(),Simple_pv.pred["MAE_4"].mean(),
                                            Simple_pv.pred["MAE_24"].mean(),Simple_pv.pred["MAE_48"].mean(),
                                            Simple_pv.pred["MAE_72"].mean(),Simple_pv.pred["MAE_96"].mean()]
        del Simple_pv

l:\Coding_project\Energy_grid_new\notebooks\XGBoost_helper.py:661: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pred_ref=df[pd.to_datetime(ref_start_t):pd.to_datetime(end_t)+timedelta(minutes=15*(K+5))].copy()


# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
File saved to:L:\Coding_project\Energy_grid_new_exp_local\PV_simple_grid_search\PV_sum_Simple_prediction_3days_0.1.csv


l:\Coding_project\Energy_grid_new\notebooks\XGBoost_helper.py:661: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pred_ref=df[pd.to_datetime(ref_start_t):pd.to_datetime(end_t)+timedelta(minutes=15*(K+5))].copy()


# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
File saved to:L:\Coding_project\Energy_grid_new_exp_local\PV_simple_grid_search\PV_sum_Simple_prediction_3days_0.2.csv


l:\Coding_project\Energy_grid_new\notebooks\XGBoost_helper.py:661: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pred_ref=df[pd.to_datetime(ref_start_t):pd.to_datetime(end_t)+timedelta(minutes=15*(K+5))].copy()


...

# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
File saved to:L:\Coding_project\Energy_grid_new_exp_local\PV_simple_grid_search\PV_sum_Simple_prediction_21days_0.3.csv


l:\Coding_project\Energy_grid_new\notebooks\XGBoost_helper.py:661: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pred_ref=df[pd.to_datetime(ref_start_t):pd.to_datetime(end_t)+timedelta(minutes=15*(K+5))].copy()


# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
File saved to:L:\Coding_project\Energy_grid_new_exp_local\PV_simple_grid_search\PV_sum_Simple_prediction_21days_0.4.csv


l:\Coding_project\Energy_grid_new\notebooks\XGBoost_helper.py:661: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pred_ref=df[pd.to_datetime(ref_start_t):pd.to_datetime(end_t)+timedelta(minutes=15*(K+5))].copy()


# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
# calculate historical average of all index in prediction duartion
# calculate pred for each step
1  done
4  done
24  done
48  done
72  done
96  done
File saved to:L:\Coding_project\Energy_grid_new_exp_local\PV_simple_grid_search\PV_sum_Simple_prediction_21days_0.5.csv


In [4]:
metrics.to_csv('metrics_simple_1.csv')